## Evals and Metaprompting

To Do:
1. o1 now takes structured outputs, so we can remove the 4o chain used to enforce a schema
2. Choose the production policy from best metaprompting result as opposed to the last policy. Current code uses a fixed loop without setting against a benchmark goal.
3. Can change model from o1-mini to o1 API via tier 5


### Initial Setup
We'll start with importing a few dependencies and setting up some visuals

In [ ]:
%pip install openai
%pip install pandas
%pip install ipython

In [11]:
from openai import OpenAI
from IPython.display import display, Markdown
import pandas as pd
from concurrent.futures import ThreadPoolExecutor
from functionDefinitions import TOOLS
import csv
import json
import os
from dotenv import load_dotenv

load_dotenv()  # Load environment variables from .env file
client = OpenAI()
MODEL = 'o1-mini'

pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

## Generating a Routine
We'll take the Flight Cancellation Policy that we have created and convert it to an LLM-based routine with the following prompt

In [13]:
# Takes human readable policy and stores it in a variable

with open('originalPolicy/flightCancellationsPolicy.md', 'r') as file:
    flight_cancellation_policy = file.read()

In [14]:
# Takes human readable policy and converts it to a AI routine

CONVERSION_PROMPT = """
You are a helpful assistant tasked with taking an external facing help center article and converting it into a internal-facing programmatically executable routine optimized for an LLM. 
The LLM using this routine will be tasked with reading the policy, answering incoming questions from customers, and helping drive the case toward resolution.

Please follow these instructions:
1. **Review the customer service policy carefully** to ensure every step is accounted for. It is crucial not to skip any steps or policies.
2. **Organize the instructions into a logical, step-by-step order**, using the specified format.
3. **Use the following format**:
   - **Main actions are numbered** (e.g., 1, 2, 3).
   - **Sub-actions are lettered** under their relevant main actions (e.g., 1a, 1b).
      **Sub-actions should start on new lines**
   - **Specify conditions using clear 'if...then...else' statements** (e.g., 'If the product was purchased within 30 days, then...').
   - **For instructions that require more information from the customer**, provide polite and professional prompts to ask for additional information.
   - **For actions that require data from external systems**, write a step to call a function using backticks for the function name (e.g., `call the check_delivery_date function`).
      - **If a step requires the customer service agent to take an action** (e.g., process a refund), generate a function call for this action (e.g., `call the process_refund function`).
      - **Define any new functions** by providing a brief description of their purpose and required parameters.
   - **If there is an action an assistant can performon behalf of the user**, include a function call for this action (e.g., `call the change_email_address function`), and ensure the function is defined with its purpose and required parameters.
      - This action may not be explicitly defined in the help center article, but can be done to help the user resolve their inquiry faster
   - **The step prior to case resolution should always be to ask if there is anything more you can assist with**.
   - **End with a final action for case resolution**: calling the `case_resolution` function should always be the final step.
4. **Ensure compliance** by making sure all steps adhere to company policies, privacy regulations, and legal requirements.
5. **Handle exceptions or escalations** by specifying steps for scenarios that fall outside the standard policy.

**Important**: If at any point you are uncertain, respond with "I don't know."

Please convert the customer service policy into the formatted routine, ensuring it is easy to follow and execute programmatically.

"""

In [19]:
def generate_routine(policy):
    try:
        messages = [
            {
                "role": "user",
                "content": f"""
                    {CONVERSION_PROMPT}

                    POLICY:
                    {policy}
                """
            }
        ]

        response = client.chat.completions.create(
            model=MODEL,
            messages=messages
        )
        

        return response.choices[0].message.content 
    except Exception as e:
        print(f"An error occurred: {e}")

In [20]:
flight_cancellation_routine = generate_routine(flight_cancellation_policy)

In [21]:
display(Markdown(flight_cancellation_routine))

```markdown
1. **Initiate Customer Interaction**
   1a. Prompt the customer: "Hello, thank you for contacting us. May I have your booking reference, name, and flight number to verify your identity?"
   1b. `call the verify_identity function` with parameters (booking_reference, name, flight_number)
      - **verify_identity Function**: Verifies the customer's identity using booking reference, name, and flight number.

2. **Determine the Nature of the Request**
   2a. Prompt the customer: "Are you looking to cancel your flight, make a change to your booking, or inquire about compensation?"
   2b. If the customer wants to **cancel**, then proceed to step 3.
   2c. If the customer wants to **change**, then proceed to step 4.
   2d. If the customer wants to **inquire about compensation**, then proceed to step 6.
   2e. Else, proceed to step 8.

3. **Handle Cancellations**
   3a. `call the check_ticket_type function` with parameter (booking_reference)
      - **check_ticket_type Function**: Determines if the ticket is refundable, non-refundable, or flexible.
   3b. If the ticket is **refundable**, then:
      3b1. `call the check_cancellation_timing function` with parameter (booking_reference)
         - **check_cancellation_timing Function**: Checks if the cancellation is within 24 hours of booking.
      3b2. If cancellation is **within 24 hours of booking**, then:
         - `call the process_full_refund function` with parameter (booking_reference)
            - **process_full_refund Function**: Processes a full refund to the original payment method.
      3b3. Else:
         - `call the determine_partial_refund function` with parameter (booking_reference)
            - **determine_partial_refund Function**: Determines the applicable refund amount based on fare rules.
         - `call the process_partial_refund function` with parameters (booking_reference, refund_amount)
            - **process_partial_refund Function**: Processes a partial refund to the original payment method.
   3c. If the ticket is **non-refundable**, then:
      3c1. Prompt the customer: "This ticket is non-refundable. Would you like to receive flight credit for future use?"
      3c2. If the customer accepts flight credit, then:
         - `call the issue_flight_credit function` with parameter (booking_reference)
            - **issue_flight_credit Function**: Issues flight credit to the customer's account, subject to an administration fee.
      3c3. Else:
         - `call the inform_penalty_fee function` with parameter (booking_reference)
            - **inform_penalty_fee Function**: Informs the customer of any applicable penalty fees.
   3d. If the cancellation is **airline-initiated**, then:
      3d1. Determine the reason for cancellation:
         - If **weather-related**:
            3d1a. Prompt the customer: "Your flight was canceled due to weather conditions. Would you like to rebook on the next available flight or receive a full refund?"
            3d1b. If rebooking is accepted, then `call the rebook_flight function` with parameter (booking_reference)
               - **rebook_flight Function**: Rebooks the customer on the next available flight without additional charges.
            3d1c. Else, `call the process_full_refund function` with parameter (booking_reference)
         - If **mechanical issues or operational changes**:
            3d1d. `call the prioritize_rebooking function` with parameter (booking_reference)
               - **prioritize_rebooking Function**: Gives priority to the customer for rebooking on the next available flight.
            3d1e. If an overnight stay is required, then `call the offer_accommodation function` with parameter (booking_reference)
               - **offer_accommodation Function**: Offers hotel accommodation to the customer.
   3e. If the cancellation is a **no-show**, then:
      3e1. If the ticket is **non-refundable**, then:
         - Prompt the customer: "No refunds are available for no-shows on non-refundable tickets. However, travel credits may be applicable. Would you like to proceed with travel credits?"
         - If accepted, `call the issue_travel_credit function` with parameter (booking_reference)
      3e2. If the ticket is **refundable**, then:
         - `call the refund_taxes_only function` with parameter (booking_reference)
            - **refund_taxes_only Function**: Refunds any applicable taxes or airport fees.

4. **Handle Changes to Booking**
   4a. `call the check_ticket_type function` with parameter (booking_reference)
   4b. If the change is **same-day**:
      4b1. If the ticket is **flexible**, then:
         - `call the change_flight_without_fee function` with parameter (booking_reference)
            - **change_flight_without_fee Function**: Allows changes to the flight without any fees, subject to availability.
      4b2. If the ticket is **non-flexible**, then:
         - `call the apply_change_fee function` with parameter (booking_reference)
            - **apply_change_fee Function**: Charges a change fee and any fare difference for the new flight.
         - Prompt the customer: "A change fee and any fare difference will apply. Do you wish to proceed?"
         - If accepted, `call the process_flight_change function` with parameters (booking_reference, new_flight_details)
   4c. If the change is **in advance**:
      4c1. If the change is **within 7 days of departure**, then:
         - `call the apply_standard_change_fee function` with parameter (booking_reference)
            - **apply_standard_change_fee Function**: Applies the standard change fee for changes within 7 days of departure.
      4c2. If the change is **beyond 7 days of departure**, then:
         - If the ticket type allows, `call the apply_lesser_or_free_change_fee function` with parameter (booking_reference)
            - **apply_lesser_or_free_change_fee Function**: Applies a lesser fee or allows free changes based on ticket type.
   4d. If the change is **airline-initiated (schedule changes)**:
      4d1. If the schedule change is **minor (less than 2 hours)**, then:
         - Prompt the customer: "Your flight schedule has changed by less than 2 hours. Would you like to accept the new schedule or request an alternative?"
         - If the customer requests an alternative, `call the offer_alternative_flight function` with parameter (booking_reference)
      4d2. If the schedule change is **major (2+ hours)**, then:
         - Prompt the customer: "Your flight schedule has changed by 2 or more hours. Would you like to rebook, receive a full refund, or opt for travel credit?"
         - If the customer chooses to rebook, `call the rebook_flight function` with parameter (booking_reference)
         - If the customer chooses a refund, `call the process_full_refund function` with parameter (booking_reference)
         - If the customer opts for travel credit, `call the issue_travel_credit function` with parameter (booking_reference)

5. **Rebooking Guidelines**
   5a. `call the get_next_available_flight function` with parameter (booking_reference)
      - **get_next_available_flight Function**: Retrieves the next available flight operated by the airline.
   5b. If a suitable flight is available with the airline, then:
      - `call the rebook_flight function` with parameter (booking_reference)
   5c. Else, `call the check_interline_partners function` with parameter (booking_reference)
      - **check_interline_partners Function**: Checks interline partners for alternative flight options.
   5d. If an alternative flight is found with a partner, then:
      - `call the rebook_with_partner function` with parameters (booking_reference, partner_flight_details)
   5e. If an upgrade is available and no economy seats are free, then:
      - Prompt the customer: "We have a complimentary upgrade available on your next flight. Would you like to accept this upgrade?"
      - If accepted, `call the apply_complimentary_upgrade function` with parameter (booking_reference)

6. **Compensation and Refund Processing**
   6a. **Refund Processing**
      6a1. If processing a refund, inform the customer: "Your refund will be processed within 7-10 business days for credit card payments or 14-20 business days for debit card payments."
      6a2. Prompt the customer: "Would you like the refund to be credited back to your original payment method or would you prefer travel credits with a 10% bonus?"
      6a3. If the customer chooses **original payment method**, then:
         - `call the process_full_refund function` with parameter (booking_reference)
      6a4. If the customer chooses **travel credits**, then:
         - `call the issue_travel_credit_with_bonus function` with parameter (booking_reference)
            - **issue_travel_credit_with_bonus Function**: Issues travel credits with a 10% bonus to the customer's account.
   
   6b. **Compensation**
      6b1. `call the determine_compensation_eligibility function` with parameters (booking_reference, delay_length, cause)
         - **determine_compensation_eligibility Function**: Determines if the customer is eligible for compensation based on delay length and cause.
      6b2. If eligible, inform the customer of their rights based on EU/US compensation rules.
      6b3. If the delay **exceeds four hours**, then:
         - `call the issue_meal_voucher function` with parameter (booking_reference)
            - **issue_meal_voucher Function**: Provides meal vouchers to the customer.
         - If an overnight stay is required, `call the offer_accommodation function` with parameter (booking_reference)
      6b4. If monetary compensation is applicable, then:
         - `call the process_monetary_compensation function` with parameter (booking_reference)

7. **Handle Special Cases**
   7a. If the customer mentions a **medical emergency**, then:
      7a1. Prompt the customer: "I'm sorry to hear that. Could you please provide a medical certificate?"
      7a2. If a medical certificate is provided, then:
         - `call the allow_flexible_cancellation function` with parameter (booking_reference)
            - **allow_flexible_cancellation Function**: Allows full cancellation or flight credit without fees.
         - `call the waive_change_fees_for_medical function` with parameter (booking_reference)
            - **waive_change_fees_for_medical Function**: Waives change fees if the customer or immediate family member is ill.
   
   7b. If the customer mentions a **bereavement case**, then:
      7b1. Prompt the customer: "I'm very sorry for your loss. Could you please provide the necessary documentation?"
      7b2. If documentation is provided, then:
         - `call the offer_bereavement_flexibility function` with parameter (booking_reference)
            - **offer_bereavement_flexibility Function**: Offers flexibility on cancellations or changes for bereavement cases.
   
   7c. If the booking is a **group booking**, then:
      7c1. If the customer wants to **partially cancel**, then:
         - `call the allow_partial_cancellation function` with parameter (booking_reference)
            - **allow_partial_cancellation Function**: Allows individual cancellations within a group booking, informing about potential impact on group rates.
      7c2. If the customer wants to **change a name**, then:
         - If the change is **7+ days before departure**, `call the permit_name_change function` with parameter (booking_reference)
            - **permit_name_change Function**: Permits one name change per passenger at no additional cost.
         - Else, inform the customer that name changes must be made 7+ days before departure.
   
   7d. If the booking involves **unaccompanied minors**, then:
      7d1. If flight changes are needed, `call the arrange_supervision function` with parameter (booking_reference)
         - **arrange_supervision Function**: Ensures proper supervision is arranged for unaccompanied minors.
      7d2. `call the prioritize_minors_rebooking function` with parameter (booking_reference)
         - **prioritize_minors_rebooking Function**: Prioritizes getting unaccompanied minors on the next available flight.

8. **Handle FAQs and Common Scenarios**
   8a. If the customer asks about **connecting flights being affected**, then:
      - `call the prioritize_missed_connection_rebooking function` with parameter (booking_reference)
         - **prioritize_missed_connection_rebooking Function**: Rebooks missed connections caused by airline delays and ensures protection for onward flights.
   
   8b. If the customer requests an **upgrade after cancellation**, then:
      - `call the offer_upgrade_based_on_availability function` with parameter (booking_reference)
         - **offer_upgrade_based_on_availability Function**: Offers upgrades based on availability or advises on fare differences.
   
   8c. If the customer wants to **change the destination**, then:
      - Prompt the customer: "Could you please provide your desired new destination?"
      - `call the check_fare_difference_and_change_fee function` with parameters (booking_reference, new_destination)
         - **check_fare_difference_and_change_fee Function**: Advises on applicable fare differences and change fees for changing the destination.
      - If the customer agrees, `call the process_destination_change function` with parameters (booking_reference, new_destination)
   
   8d. If the booking is a **third-party booking**, then:
      - Prompt the customer: "Is your booking made through a travel agency or online platform?"
      - If yes, prompt: "Please contact your original booking agent to make changes. I can assist if your booking is eligible for direct airline handling."
      - If eligible, `call the assist_direct_airline_handling function` with parameter (booking_reference)
   
   8e. If the customer **requests compensation for inconvenience**, then:
      - `call the assess_compensation_eligibility function` with parameter (booking_reference)
         - **assess_compensation_eligibility Function**: Evaluates eligibility based on the airline's compensation policy.
      - If eligible, inform the customer about available compensation options like meal vouchers, accommodation, or monetary compensation.
      - `call the provide_compensation_options function` with parameter (booking_reference)
   
   8f. If the customer's **preferred flight is fully booked**, then:
      - `call the offer_next_available_flight function` with parameter (booking_reference)
         - **offer_next_available_flight Function**: Offers the next available flight option and places the customer on a waitlist for their preferred flight if possible.
      - If an upgrade is available, prompt the customer: "Your preferred flight is fully booked. Would you like to upgrade to a higher class if available?"
      - If accepted, `call the apply_available_upgrade function` with parameter (booking_reference)

9. **Final Assistance and Case Resolution**
   9a. Prompt the customer: "Is there anything else I can assist you with today?"
   9b. `call the case_resolution function` with parameter (booking_reference)
      - **case_resolution Function**: Finalizes the case and ensures all customer requests have been addressed.
```

## Evaluating Accuracy

Now that we have a routine generated with o1, we can run it against our evaluation suite and measure its accuracy.

We'll start by creating an agent that is equipped with the policy and a list of tools. It will be given messages from an existing conversation and will be tasked with determining the next best action to take

In [22]:
def agent_response(transcript, policy, model):
    try:
        messages = [
            {
                "role": "system",
                "content": f"""
You are a customer service agent that is responsible for handling airline related issues. Below is the exact policy that you must follow to address the customer's issue

POLICY:
{policy}
                """
            }
        ]

        messages.extend(transcript)
        response = client.chat.completions.create(
            model=model,
            messages=messages,
            tools=TOOLS
        )
        
        return response.choices[0].message 
    except Exception as e:
        print(f"An error occurred: {e}")

We will process each row in parallel to reduce runtime and compare the function call + inputs that the model selects against our expected function + parameters.

In [23]:
def process_row(row_number, row, policy, model):
    try:
        # Extract values from the current row
        conversation_str = row['conversation']
        expected_function = row['expected_function']
        expected_inputs_str = row['expected_inputs']

        # Parse the conversation JSON
        try:
            conversation = json.loads(conversation_str)
        except json.JSONDecodeError as e:
            print(f"Error parsing 'conversation' in row {row_number}: {e}")
            conversation = None

        # Parse the expected_inputs JSON
        try:
            expected_inputs = json.loads(expected_inputs_str)
            # If expected_inputs is a string (double-encoded), parse it again
            if isinstance(expected_inputs, str):
                expected_inputs = json.loads(expected_inputs)
        except json.JSONDecodeError as e:
            print(f"Error parsing 'expected_inputs' in row {row_number}: {e}")
            expected_inputs = None

        # Extract the last assistant's message content if it exists
        response = agent_response(conversation, policy, model)
        assistant_message_content = response.content if response else None
        tool_calls = response.tool_calls

        # If the most recent response does not contain a tool call and just a message from the assistant, we rerun it once more to get our tool call.
        if not tool_calls:
            if assistant_message_content:
                # Append the assistant's message content to the conversation
                conversation.append({"role": "assistant", "content": assistant_message_content})
                # Make another call to agent_response
                response = agent_response(conversation, policy, model)
                tool_calls = response.tool_calls

        if not tool_calls:
            actual_function = None
            actual_inputs = None
            is_correct = False
        else:
            tool_call = tool_calls[0]  # Assuming we're only interested in the first tool call
            function_name = tool_call.function.name
            arguments = json.loads(tool_call.function.arguments)

            is_correct = (function_name == expected_function) and (arguments == expected_inputs)
            actual_function = function_name
            actual_inputs = arguments

        return {
            'expected_function': expected_function,
            'expected_inputs': expected_inputs,
            'actual_function': actual_function,
            'actual_inputs': actual_inputs,
            'is_correct': is_correct,
            'assistant_message_content': assistant_message_content
        }

    except Exception as e:
        print(f"Error processing row {row_number}: {e}")
        return {
            'expected_function': row.get('expected_function'),
            'expected_inputs': row.get('expected_inputs'),
            'actual_function': None,
            'actual_inputs': None,
            'is_correct': False,
            'assistant_message_content': None
        }

def evaluate_function_calls(file_path, policy, model):
    records = []

    # Check if the file exists
    if not os.path.isfile(file_path):
        print(f"Error: The file '{file_path}' does not exist.")
        return

    try:
        with open(file_path, 'r', newline='', encoding='utf-8') as csvfile:
            # Initialize the CSV reader with pipe as delimiter
            reader = csv.DictReader(csvfile, delimiter='|', quotechar='"', escapechar='\\')

            # Use ThreadPoolExecutor to process rows in parallel
            with ThreadPoolExecutor() as executor:
                futures = {executor.submit(process_row, row_number, row, policy, model): row_number for row_number, row in enumerate(reader, start=2)}
                for future in futures:
                    record = future.result()
                    records.append(record)

    except Exception as e:
        print(f"An unexpected error occurred while reading the CSV file: {e}")
        return

    df = pd.DataFrame(records)
    total_accuracy = df['is_correct'].mean()
    return df, total_accuracy


Let's take a look at our results.

In [9]:
# Assuming the CSV file is located at 'evals/functionCallingEval.csv'
df, accuracy = evaluate_function_calls('evals/functionCallingEval.csv', flight_cancellation_routine, 'gpt-4o-mini-2024-07-18')

# Display the accuracy as a mini header
display(Markdown(f"### Accuracy: {accuracy:.2%}"))

display(df)

### Accuracy: 70.59%

,expected_function,expected_inputs,actual_function,actual_inputs,is_correct,assistant_message_content
0,check_ticket_type,{'booking_reference': 'ABC123'},check_ticket_type,{'booking_reference': 'ABC123'},True,None
1,check_ticket_type,{'booking_reference': 'DEF456'},check_ticket_type,{'booking_reference': 'DEF456'},True,None
2,process_full_refund,{'booking_reference': 'GHI789'},check_ticket_type,{'booking_reference': 'GHI789'},False,None
3,offer_flight_credit,{'booking_reference': 'JKL012'},offer_flight_credit,{'booking_reference': 'JKL012'},True,"You have a non-refundable ticket. I can provide you with flight credit for future use, subject to an administration fee. Additionally, please note that a penalty fee applies to non-refundable tickets.\n\nWould you like me to proceed with offering you flight credit, or do you need any further information?"
4,check_ticket_type,{'booking_reference': 'MNO345'},check_ticket_type,{'booking_reference': 'MNO345'},True,None
5,prioritize_missed_connections,{'booking_reference': 'PQR678'},prioritize_missed_connections,{'booking_reference': 'PQR678'},True,None
6,check_compensation_eligibility,{'booking_reference': 'STU901'},check_compensation_eligibility,{'booking_reference': 'STU901'},True,None
7,process_change_no_fee,{'booking_reference': 'VWX234'},None,None,False,Would you like to make a same-day change or a change in advance?
8,process_flexible_cancellation,"{'booking_reference': 'YZA567', 'medical_certificate': 'medical_certificate_001'}",process_flexible_cancellation,"{'booking_reference': 'YZA567', 'medical_certificate': 'medical_certificate_001'}",True,None
9,permit_name_change,{'booking_reference': 'BCD890'},process_partial_group_cancellation,{'booking_reference': 'BCD890'},False,None


## Metaprompting

Let's now leverage o1 again to add in a metaprompting loop to see if we can improve the quality of our evals.

We'll take the following multi-step approach:
- We'll pass in the current routine + eval results to o1 and ask it analyze the results and update the routine accordingly
- Since o1 does not currently support structured outputs, we'll chain with output with a 4o to enforce a schema we can parse
- Finally, we take the new routine and run it back through our eval to generate new results

We'll run this loop a fixed number of times and see what improvements we can make

In [24]:
def metaprompt(messages):
    try:
        response = client.chat.completions.create(
            model='o1-preview',
            messages=messages,
        )
        
        return response.choices[0].message.content
    except Exception as e:
        print(f"An error occurred: {e}")

In [25]:
def enforce_schema(updated_prompt):
    try:
        messages = [
            {
                "role": "system",
                "content": f"""
You will be given a response from an LLM that just generated a policy for flight cancellations.
Your task is to take just the policy exactly as it is written and return it in the defined json. Remove all parts from the LLM's answer that are not part of the policy.

LLM RESPONSE:
{updated_prompt}
                """
            }
        ]

        response = client.chat.completions.create(
            model='gpt-4o-2024-08-06',
            messages=messages,
            response_format= {
                "type": "json_schema",
                "json_schema": {
                    "name": "policy_output",
                    "schema": {
                    "type": "object",
                    "properties": {
                        "final_answer": { "type": "string" }
                    },
                    "required": ["final_answer"],
                    "additionalProperties": False
                    },
                    "strict": True
                }
            }

        )
        
        return response.choices[0].message.content
    except Exception as e:
        print(f"An error occurred: {e}") 

In [26]:
updated_policy = flight_cancellation_routine
messages = [
    {
        "role": "user",
        "content": f"""
You are an agent that is responsible for improving the quality of routine instructions that are provided to a customer service LLM agent.

I am going to give you the policy for the customer service agent that contains detailed instructions on how to handle flight cancellations and changes.

You will also be provided with the results from an eval set that include the following:
    - conversation history: This is the conversation that we present to the LLM along with the system prompt
    - expected_function: This is the function we expect the LLM to call
    - expected_input: This is the input we expect the LLM to provide to the function
    - actual_function: This is the actual function the LLM called
    - actual_input: This is the actual input the LLM provided
    - assistant_message_content: This is the message the LLM generated when it returned its response
    - is_correct: True/False value depending on if the model responded correctly

Carefully analyze the instructions provided as well as the results of the eval. Get a firm understanding of the failures in the policy.

Return an updated policy that will perform better against the dataset.

Here is the current policy:
{flight_cancellation_policy}
"""
    }
]

for _ in range(5):
    # Evaluate the function calls with the current policy
    df, accuracy = evaluate_function_calls('evals/functionCallingEval.csv', updated_policy, 'gpt-4o-mini-2024-07-18')
    
    # Display the accuracy as a mini header
    display(Markdown(f"### Accuracy: {accuracy:.2%}"))
    display(df)
    results_json = df.to_json(orient='records')

    messages.append({
        "role": "user",
        "content": f"""
Here are the results based on the current policy:
{results_json}
"""
    })
    # Use the metaprompt function to get an updated policy
    temp_policy_json = enforce_schema(metaprompt(messages))
    temp_policy_str = temp_policy_json.strip("json```").strip("```")
    temp_policy = json.loads(temp_policy_str)["final_answer"]
    print(f"Corrected Policy: {temp_policy}")

    messages.append({
        "role": "assistant",
        "content": f"""
{temp_policy}
"""
    })

    # Update the policy for the next iteration
    updated_policy = temp_policy


### Accuracy: 88.24%

,expected_function,expected_inputs,actual_function,actual_inputs,is_correct,assistant_message_content
0,check_ticket_type,{'booking_reference': 'ABC123'},check_ticket_type,{'booking_reference': 'ABC123'},True,None
1,check_ticket_type,{'booking_reference': 'DEF456'},check_ticket_type,{'booking_reference': 'DEF456'},True,None
2,process_full_refund,{'booking_reference': 'GHI789'},process_full_refund,{'booking_reference': 'GHI789'},True,None
3,offer_flight_credit,{'booking_reference': 'JKL012'},None,None,False,Your ticket is non-refundable. Would you like to receive flight credit for future use?
4,check_ticket_type,{'booking_reference': 'MNO345'},check_ticket_type,{'booking_reference': 'MNO345'},True,None
5,prioritize_missed_connections,{'booking_reference': 'PQR678'},prioritize_missed_connections,{'booking_reference': 'PQR678'},True,None
6,check_compensation_eligibility,{'booking_reference': 'STU901'},check_compensation_eligibility,{'booking_reference': 'STU901'},True,None
7,process_change_no_fee,{'booking_reference': 'VWX234'},process_change_no_fee,{'booking_reference': 'VWX234'},True,"Your ticket is flexible, which means you can change your flight without any fees, subject to availability. Please provide me with the details of the new flight you would like to book?"
8,process_flexible_cancellation,"{'booking_reference': 'YZA567', 'medical_certificate': 'medical_certificate_001'}",process_flexible_cancellation,"{'booking_reference': 'YZA567', 'medical_certificate': 'medical_certificate_001'}",True,None
9,permit_name_change,{'booking_reference': 'BCD890'},permit_name_change,{'booking_reference': 'BCD890'},True,None


Corrected Policy: **Updated Internal Flight Cancellations and Changes Policy**

*This updated policy incorporates explicit instructions on the procedural steps and the appropriate functions to use in various scenarios, addressing the issues identified in the evaluation.*

---

**Purpose**: This document serves as a detailed guide for internal support agents to handle flight cancellations and changes. The focus is on providing clear instructions, ensuring efficiency, consistency, and customer satisfaction during customer interactions.

**Note**: Always maintain a calm, empathetic tone while assisting customers, especially during stressful situations involving cancellations or major changes.

---

## **Table of Contents**

1. General Guidelines for Handling Customer Requests  
2. Cancellations: Types and Policies  
3. Changes: Types and Policies  
4. Rebooking Guidelines  
5. Compensation and Refund Rules  
6. Special Cases  
7. FAQs for Common Scenarios

---

### **1\. General Guideline

### Accuracy: 94.12%

,expected_function,expected_inputs,actual_function,actual_inputs,is_correct,assistant_message_content
0,check_ticket_type,{'booking_reference': 'ABC123'},check_ticket_type,{'booking_reference': 'ABC123'},True,None
1,check_ticket_type,{'booking_reference': 'DEF456'},check_ticket_type,{'booking_reference': 'DEF456'},True,None
2,process_full_refund,{'booking_reference': 'GHI789'},process_full_refund,{'booking_reference': 'GHI789'},True,None
3,offer_flight_credit,{'booking_reference': 'JKL012'},offer_flight_credit,{'booking_reference': 'JKL012'},True,"Since you have a non-refundable ticket, I'll proceed to offer you flight credit for future use, which may be subject to an administration fee. Let me initiate that for you now."
4,check_ticket_type,{'booking_reference': 'MNO345'},check_ticket_type,{'booking_reference': 'MNO345'},True,None
5,prioritize_missed_connections,{'booking_reference': 'PQR678'},prioritize_missed_connections,{'booking_reference': 'PQR678'},True,None
6,check_compensation_eligibility,{'booking_reference': 'STU901'},check_compensation_eligibility,{'booking_reference': 'STU901'},True,None
7,process_change_no_fee,{'booking_reference': 'VWX234'},None,None,False,"You have a flexible ticket, which allows changes without any fees, subject to availability. Please provide the new flight details you'd like to rebook to."
8,process_flexible_cancellation,"{'booking_reference': 'YZA567', 'medical_certificate': 'medical_certificate_001'}",process_flexible_cancellation,"{'booking_reference': 'YZA567', 'medical_certificate': 'medical_certificate_001'}",True,None
9,permit_name_change,{'booking_reference': 'BCD890'},permit_name_change,{'booking_reference': 'BCD890'},True,None


...

,expected_function,expected_inputs,actual_function,actual_inputs,is_correct,assistant_message_content
0,check_ticket_type,{'booking_reference': 'ABC123'},check_ticket_type,{'booking_reference': 'ABC123'},True,None
1,check_ticket_type,{'booking_reference': 'DEF456'},check_ticket_type,{'booking_reference': 'DEF456'},True,None
2,process_full_refund,{'booking_reference': 'GHI789'},process_full_refund,{'booking_reference': 'GHI789'},True,None
3,offer_flight_credit,{'booking_reference': 'JKL012'},offer_flight_credit,{'booking_reference': 'JKL012'},True,Your ticket is classified as non-refundable. I will offer you flight credit for future use. Please hold on for a moment while I process that.
4,check_ticket_type,{'booking_reference': 'MNO345'},check_ticket_type,{'booking_reference': 'MNO345'},True,None
5,prioritize_missed_connections,{'booking_reference': 'PQR678'},prioritize_missed_connections,{'booking_reference': 'PQR678'},True,None
6,check_compensation_eligibility,{'booking_reference': 'STU901'},check_compensation_eligibility,{'booking_reference': 'STU901'},True,None
7,process_change_no_fee,{'booking_reference': 'VWX234'},None,None,False,"Your ticket is flexible, which means you can make changes without any fees. Could you please provide the new flight details you would like to book?"
8,process_flexible_cancellation,"{'booking_reference': 'YZA567', 'medical_certificate': 'medical_certificate_001'}",process_flexible_cancellation,"{'booking_reference': 'YZA567', 'medical_certificate': 'medical_certificate_001'}",True,None
9,permit_name_change,{'booking_reference': 'BCD890'},permit_name_change,{'booking_reference': 'BCD890'},True,None


Corrected Policy: # **Updated Internal Flight Cancellations and Changes Policy**

*This updated policy addresses the issues identified in the latest evaluation by clarifying the procedural steps when additional customer input is required. Assistants are instructed to call the appropriate functions even when awaiting further details, ensuring better compliance and operational efficiency.*

---

## **Summary of Key Updates**

- **Mandatory Function Calls with Pending Inputs**: Assistants must call the appropriate function immediately after determining the required action, even if additional customer input is needed. Inputs that are not yet available should be passed as `null` or placeholders like "pending".

- **Updated Function Definitions**: Functions now accommodate cases where some parameters may be incomplete or awaiting customer input.

- **Explicit Procedural Steps**: Clear instructions and examples are provided for scenarios where customer input is needed to complete a process.



### Accuracy: 82.35%

,expected_function,expected_inputs,actual_function,actual_inputs,is_correct,assistant_message_content
0,check_ticket_type,{'booking_reference': 'ABC123'},check_ticket_type,{'booking_reference': 'ABC123'},True,None
1,check_ticket_type,{'booking_reference': 'DEF456'},check_ticket_type,{'booking_reference': 'DEF456'},True,None
2,process_full_refund,{'booking_reference': 'GHI789'},process_full_refund,{'booking_reference': 'GHI789'},True,None
3,offer_flight_credit,{'booking_reference': 'JKL012'},check_fare_rules,{'booking_reference': 'JKL012'},False,Your ticket is non-refundable. I will now check the fare rules to determine what options are available for your cancellation.
4,check_ticket_type,{'booking_reference': 'MNO345'},check_ticket_type,{'booking_reference': 'MNO345'},True,None
5,prioritize_missed_connections,{'booking_reference': 'PQR678'},prioritize_missed_connections,{'booking_reference': 'PQR678'},True,None
6,check_compensation_eligibility,{'booking_reference': 'STU901'},check_compensation_eligibility,{'booking_reference': 'STU901'},True,None
7,process_change_no_fee,{'booking_reference': 'VWX234'},process_change_no_fee,"{'booking_reference': 'VWX234', 'new_flight_details': None}",False,"Your ticket is flexible, which means you can make changes without any fees. Please provide the new flight details you would like to book. \n\nI'll start the change process while we wait for your details."
8,process_flexible_cancellation,"{'booking_reference': 'YZA567', 'medical_certificate': 'medical_certificate_001'}",process_flexible_cancellation,"{'booking_reference': 'YZA567', 'medical_certificate': 'medical_certificate_001'}",True,None
9,permit_name_change,{'booking_reference': 'BCD890'},permit_name_change,{'booking_reference': 'BCD890'},True,None


Corrected Policy: ### **1\. General Guidelines for Handling Customer Requests**

- **Confirm Identity**:  
  - **Function to Use**: `verify_customer_identity(booking_reference, customer_details)`  
  - **Action**: Verify the customer's identity by asking for their booking reference and required personal details.

- **Listen and Understand**:  
  - **Action**: Determine the customer's request—whether they want to cancel, change, or inquire about compensation.

- **Check the Ticket Type**:  
  - **Function to Use**: `check_ticket_type(booking_reference)`  
  - **Action**: Identify if the ticket is refundable, non-refundable, or flexible.

- **Follow Procedural Steps**:  
  - **Action**: Adhere strictly to the procedural steps outlined in this policy, calling the correct functions at each stage.

- **Customer Communication**:  
  - **Action**: Maintain clear and empathetic communication, informing the customer of actions being taken and requesting any additional information needed.

- **F

## Distilling Down to a smaller model

Each time we release a new snapshot of a model, it is always a challenge to ensure that your existing prompt works for the new snapshot.

In this example, we'll simulate that work by trying to get the routine to work for our older GPT 3.5 Turbo model.

In [28]:
messages = [
    {
        "role": "user",
        "content": f"""
You are an agent that is responsible for improving the quality of routine instructions that are provided to a customer service LLM agent.

I am going to give you the policy for the customer service agent that contains detailed instructions on how to handle flight cancellations and changes.

You will also be provided with the results from an eval set that include the following:
    - conversation history: This is the conversation that we present to the LLM along with the system prompt
    - expected_function: This is the function we expect the LLM to call
    - expected_input: This is the input we expect the LLM to provide to the function
    - actual_function: This is the actual function the LLM called
    - actual_input: This is the actual input the LLM provided
    - assistant_message_content: This is the message the LLM generated when it returned its response
    - is_correct: True/False value depending on if the model responded correctly

Carefully analyze the instructions provided as well as the results of the eval. Get a firm understanding of the failures in the policy.

Return an updated policy that will perform better against the dataset.

Here is the current policy:
{updated_policy}
"""
    }
]

for _ in range(5):
    # Evaluate the function calls with the current policy
    df, accuracy = evaluate_function_calls('evals/functionCallingEval.csv', updated_policy, 'gpt-4o-mini-2024-07-18')
    
    # Display the accuracy as a mini header
    display(Markdown(f"### Accuracy: {accuracy:.2%}"))
    display(df)

    results_json = df.to_json(orient='records')

    messages.append({
        "role": "user",
        "content": f"""
Here are the results based on the current policy:
{results_json}
"""
    })
    # Use the metaprompt function to get an updated policy
    temp_policy_json = enforce_schema(metaprompt(messages))
    temp_policy_str = temp_policy_json.strip("json```").strip("```")
    temp_policy = json.loads(temp_policy_str)["final_answer"]
    print(f"Corrected Policy: {temp_policy}")

    messages.append({
        "role": "assistant",
        "content": f"""
{temp_policy}
"""
    })

    # Update the policy for the next iteration
    updated_policy = temp_policy


### Accuracy: 70.59%

,expected_function,expected_inputs,actual_function,actual_inputs,is_correct,assistant_message_content
0,check_ticket_type,{'booking_reference': 'ABC123'},None,None,False,Can you please let me know if the cancellation is within 24 hours of your booking?
1,check_ticket_type,{'booking_reference': 'DEF456'},check_ticket_type,{'booking_reference': 'DEF456'},True,None
2,process_full_refund,{'booking_reference': 'GHI789'},process_full_refund,{'booking_reference': 'GHI789'},True,None
3,offer_flight_credit,{'booking_reference': 'JKL012'},None,None,False,Your ticket is non-refundable. I can offer you flight credit for future use. Would you like to proceed with converting your booking to flight credit?
4,check_ticket_type,{'booking_reference': 'MNO345'},check_ticket_type,{'booking_reference': 'MNO345'},True,None
5,prioritize_missed_connections,{'booking_reference': 'PQR678'},prioritize_missed_connections,{'booking_reference': 'PQR678'},True,None
6,check_compensation_eligibility,{'booking_reference': 'STU901'},check_compensation_eligibility,{'booking_reference': 'STU901'},True,None
7,process_change_no_fee,{'booking_reference': 'VWX234'},None,None,False,"Your ticket is flexible, which means you can change your flight without any fees. Please provide the new flight details you'd like to book."
8,process_flexible_cancellation,"{'booking_reference': 'YZA567', 'medical_certificate': 'medical_certificate_001'}",process_flexible_cancellation,"{'booking_reference': 'YZA567', 'medical_certificate': 'medical_certificate_001'}",True,None
9,permit_name_change,{'booking_reference': 'BCD890'},permit_name_change,{'booking_reference': 'BCD890'},True,None


Corrected Policy: ### **2. Cancellations: Types and Policies**

#### **2.1 Customer-Initiated Cancellations Within 24 Hours of Booking**

**Procedural Steps**:

1. **Verify Customer Identity**
   - `verify_customer_identity(booking_reference, customer_details)`

2. **Retrieve Booking Details**
   - `get_booking_details(booking_reference)`

3. **Check if Within 24 Hours of Booking**
   - **Action**: Calculate if the cancellation request is within 24 hours of the original booking time.
   - **Note**: Do **not** ask the customer if the booking was made within 24 hours; use available data.

4. **Proceed to Process Full Refund**
   - **Function to Use**: `process_full_refund(booking_reference)`
   - **Customer Communication**:
     - Inform the customer: "Since you've requested to cancel within 24 hours of your booking, we can process a full refund for you."

#### **2.2 Customer-Initiated Cancellations Beyond 24 Hours**

**Procedural Steps**:

1. **Verify Customer Identity**
   - `verify_cu

### Accuracy: 82.35%

,expected_function,expected_inputs,actual_function,actual_inputs,is_correct,assistant_message_content
0,check_ticket_type,{'booking_reference': 'ABC123'},check_ticket_type,{'booking_reference': 'ABC123'},True,None
1,check_ticket_type,{'booking_reference': 'DEF456'},check_ticket_type,{'booking_reference': 'DEF456'},True,None
2,process_full_refund,{'booking_reference': 'GHI789'},process_full_refund,{'booking_reference': 'GHI789'},True,None
3,offer_flight_credit,{'booking_reference': 'JKL012'},None,None,False,Your ticket is non-refundable. I can offer you flight credit for future use. Would you like to proceed with converting your booking into flight credit?
4,check_ticket_type,{'booking_reference': 'MNO345'},check_ticket_type,{'booking_reference': 'MNO345'},True,None
5,prioritize_missed_connections,{'booking_reference': 'PQR678'},prioritize_missed_connections,{'booking_reference': 'PQR678'},True,None
6,check_compensation_eligibility,{'booking_reference': 'STU901'},check_compensation_eligibility,{'booking_reference': 'STU901'},True,None
7,process_change_no_fee,{'booking_reference': 'VWX234'},rebook_without_fee,{'booking_reference': 'VWX234'},False,None
8,process_flexible_cancellation,"{'booking_reference': 'YZA567', 'medical_certificate': 'medical_certificate_001'}",process_flexible_cancellation,"{'booking_reference': 'YZA567', 'medical_certificate': 'medical_certificate_001'}",True,None
9,permit_name_change,{'booking_reference': 'BCD890'},permit_name_change,{'booking_reference': 'BCD890'},True,None


...

,expected_function,expected_inputs,actual_function,actual_inputs,is_correct,assistant_message_content
0,check_ticket_type,{'booking_reference': 'ABC123'},check_ticket_type,{'booking_reference': 'ABC123'},True,None
1,check_ticket_type,{'booking_reference': 'DEF456'},check_ticket_type,{'booking_reference': 'DEF456'},True,None
2,process_full_refund,{'booking_reference': 'GHI789'},process_refund,"{'booking_reference': 'GHI789', 'original_payment_method': 'credit card'}",False,None
3,offer_flight_credit,{'booking_reference': 'JKL012'},offer_flight_credit,{'booking_reference': 'JKL012'},True,"Your ticket is non-refundable. Unfortunately, we cannot process a refund, but I can offer you flight credit for future travel. Would you like to proceed with the flight credit?"
4,check_ticket_type,{'booking_reference': 'MNO345'},check_ticket_type,{'booking_reference': 'MNO345'},True,None
5,prioritize_missed_connections,{'booking_reference': 'PQR678'},prioritize_missed_connections,{'booking_reference': 'PQR678'},True,None
6,check_compensation_eligibility,{'booking_reference': 'STU901'},check_compensation_eligibility,{'booking_reference': 'STU901'},True,None
7,process_change_no_fee,{'booking_reference': 'VWX234'},None,None,False,"Your ticket is flexible, so you can change your flight without any fees. Please provide the new flight details you'd like to book."
8,process_flexible_cancellation,"{'booking_reference': 'YZA567', 'medical_certificate': 'medical_certificate_001'}",process_flexible_cancellation,"{'booking_reference': 'YZA567', 'medical_certificate': 'medical_certificate_001'}",True,None
9,permit_name_change,{'booking_reference': 'BCD890'},permit_name_change,{'booking_reference': 'BCD890'},True,None


Corrected Policy: ### **1. General Guidelines for Handling Customer Requests**

- **Confirm Identity**:
  - **Function to Use**: `verify_customer_identity(booking_reference, customer_details)`
  - **Action**: Verify the customer's identity by asking for their booking reference and required personal details.

- **Listen and Understand**:
  - **Action**: Determine the customer's request—whether they want to cancel, change, report a missed connection, request a name change, or inquire about compensation.

- **Retrieve Booking Details**:
  - **Function to Use**: `get_booking_details(booking_reference)`
  - **Action**: Retrieve all necessary booking information after verifying identity.

- **Check the Ticket Type**:
  - **Function to Use**: `check_ticket_type(booking_reference)`
  - **Action**: Identify if the ticket is refundable, non-refundable, flexible, or has special conditions.

- **Follow Procedural Steps**:
  - **Action**: Adhere strictly to the procedural steps outlined in this pol

### Accuracy: 76.47%

,expected_function,expected_inputs,actual_function,actual_inputs,is_correct,assistant_message_content
0,check_ticket_type,{'booking_reference': 'ABC123'},check_ticket_type,{'booking_reference': 'ABC123'},True,None
1,check_ticket_type,{'booking_reference': 'DEF456'},check_ticket_type,{'booking_reference': 'DEF456'},True,None
2,process_full_refund,{'booking_reference': 'GHI789'},process_refund,"{'booking_reference': 'GHI789', 'original_payment_method': 'credit card'}",False,None
3,offer_flight_credit,{'booking_reference': 'JKL012'},offer_flight_credit,{'booking_reference': 'JKL012'},True,"Your ticket is non-refundable. Unfortunately, we cannot process a refund, but we can offer you flight credit for future use.\n\nWould you like to proceed with that?"
4,check_ticket_type,{'booking_reference': 'MNO345'},check_ticket_type,{'booking_reference': 'MNO345'},True,None
5,prioritize_missed_connections,{'booking_reference': 'PQR678'},prioritize_missed_connections,{'booking_reference': 'PQR678'},True,None
6,check_compensation_eligibility,{'booking_reference': 'STU901'},check_compensation_eligibility,{'booking_reference': 'STU901'},True,None
7,process_change_no_fee,{'booking_reference': 'VWX234'},None,None,False,"You have a flexible ticket, which means you can change your flight without any fees. Please provide the new flight details you'd like to book."
8,process_flexible_cancellation,"{'booking_reference': 'YZA567', 'medical_certificate': 'medical_certificate_001'}",process_flexible_cancellation,"{'booking_reference': 'YZA567', 'medical_certificate': 'medical_certificate_001'}",True,None
9,permit_name_change,{'booking_reference': 'BCD890'},permit_name_change,{'booking_reference': 'BCD890'},True,None


Corrected Policy: ### **2. Cancellations: Types and Policies**

#### **2.1 Customer-Initiated Cancellations Within 24 Hours of Booking**

**Procedural Steps**:

1. **Verify Customer Identity**
   - `verify_customer_identity(booking_reference, customer_details)`

2. **Retrieve Booking Details**
   - `get_booking_details(booking_reference)`

3. **Check if Within 24 Hours of Booking**
   - **Action**: Determine using booking details if the cancellation request is within 24 hours of the original booking time.

4. **Proceed to Process Full Refund**
   - **Function to Use**: `process_full_refund(booking_reference)`
   - **Customer Communication**:
     - Inform the customer: "Since you've requested to cancel within 24 hours of your booking, we can process a full refund for you."
   - **Action**: Call `process_full_refund(booking_reference)` immediately after informing the customer.

#### **2.2 Customer-Initiated Cancellations Beyond 24 Hours**

**Procedural Steps**:

1. **Verify Customer Ide